<a href="https://colab.research.google.com/github/carlomigs/btgym/blob/master/examples/guided_a3c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Kismuz/btgym.git

Cloning into 'btgym'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6981 (delta 0), reused 0 (delta 0), pack-reused 6977
Receiving objects: 100% (6981/6981), 126.68 MiB | 22.64 MiB/s, done.
Resolving deltas: 100% (5168/5168), done.
Checking out files: 100% (346/346), done.


In [3]:
%cd btgym

/content/btgym


In [5]:
!ls

btgym  docs  examples  img  LICENSE  README.md	setup.py  tests


In [6]:
pip install -e '/content/btgym'

Obtaining file:///content/btgym
     |████████████████████████████████| 419kB 9.6MB/s 
     |████████████████████████████████| 14.6MB 37.7MB/s 
     |████████████████████████████████| 92kB 31.0MB/s 
  Created wheel for logbook: filename=Logbook-1.5.2-cp36-cp36m-linux_x86_64.whl size=66346 sha256=d3f7c729ac07ed8e2d6b2d9db1e1e8c671d21f11fd25a6c20630262746eb93c5
  Stored in directory: /root/.cache/pip/wheels/c7/d6/22/9f6bd2884a48b215fdd9cbfa78671b51f4470dfd3d14ef701a
Successfully built logbook
ERROR: plotnine 0.5.1 has requirement matplotlib>=3.0.0, but you'll have matplotlib 2.0.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.0.3
    Uninstalling matplotlib-3.0.3:
      Successfully uninstalled matplotlib-3.0.3
  Running setup.py develop for btgym


### Guided policy search 

_Speeding up training by applying auxillary imitation loss on expert actions:_

`L_gps = lamda_a3c * L_a3c + guided_lambda * L_gps`

##### Intuition:
- This implementation is loosely reffered as 'guided policy search' after algorithm described in paper by S. Levine and P. Abbeel [Learning Neural Network Policies with Guided PolicySearch under Unknown Dynamics](https://people.eecs.berkeley.edu/~svlevine/papers/mfcgps.pdf) in a sense that we utilize same idea of fitting 'local' (here - single episode) oracle for environment with unknown dynamics and use actions demonstrated by it to optimize trajectory distribution for training agent; also  connected to s RLfD ideas.

- Using expert actions is proven way to speed up training by exploring more relevant state-action space regiones;

- It can also lead to suboptimal policies when demonstrated actions are suboptimal (learnt model is irrelevant) and agent is strictly entingled   to expert trajectories (cannot explore and act on it's own);
- So it is essential to find ballanced degree of agent dependensy on advisor;

##### Papers:
- Levine et al., [Learning Neural Network Policies with Guided PolicySearch under Unknown Dynamics](https://people.eecs.berkeley.edu/~svlevine/papers/mfcgps.pdf)

- Brys et al., [Reinforcement Learning from Demonstration through Shaping](https://www.ijcai.org/Proceedings/15/Papers/472.pdf)

- Wiewiora et al., [Principled Methods for Advising Reinforcement Learning Agents](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.14.6412&rep=rep1&type=pdf)

- Abbeel, Ng, [Exploration and Apprenticeship Learning in Reinforcement Learning](https://icml.cc/Conferences/2005/proceedings/papers/001_Exploration_AbbeelNg.pdf)

##### Implementation details:

- For each train episode expert has access to entire data range but does not provides complete state-action trajectories. Instead it fits simple local model of `external` part of the environment which provides  `advises` in form of categorical probability distribution in action space. Morover, imitation loss is defined in actions subspace, namely over 'buy' and 'sell' actions. Such relaxed conditions seems to work better than strictly following [possibly suboptimal] expert trajectory.

- Note that expert advices conditioned on external state observations only, i.e. price dynamics; expert doesn't account current inner agent state (opened position, acoount value etc.). It can be said it acts similar to real-life financial advisor'...now it probably time to buy'; it does not bear any responsibility for advices given so it is on agent's side to deside follow or not. The degree of agent 'independency' from oracle is regulated by `guided_lambda` hyperparamter. When set to zero - oracle advises completely ignored. When set >> 1 it can be clearly seen (with sine wave data for example) than agent performace is degraded due to strong dependency from oracle; reasonable values for this setup seems to lie  within 0.1 .. 1.0 range;

##### Expert estimated actions example:
- currently oracle implements extremely simple strategy: it finds local peaks along entire given episode data, filters it by time and value and estimate signals by reversing position after every up- or down- peak. Than smoothes by convolving signals with gaussian kernel, normalises and outputs discrete prob. distribution over action space.

![Signal](https://github.com/carlomigs/btgym/blob/master/examples/img/oracle_signal.png?raw=1)

##### Expert actions distribution:

- It seems that using MSE error on action subsets leaves agent better degree of freedom than cross-entropy loss over entire action space; though it should be highly dependend on quality of advised actions.

![A.Distr](https://github.com/carlomigs/btgym/blob/master/examples/img/oracle_p_distr.png?raw=1)

##### Example of distribution over actions learnt by agent *:
*sample, irrelevant to image above
![AgentA.Distr](https://github.com/carlomigs/btgym/blob/master/examples/img/learnt_a_p_d.png?raw=1)

### Addition for those who like to look: extended summaries.
- Tensorboard summaries (see images tab) are updated and now include:

##### - action probabilities for an episode (see above);

##### - value function for an episode:
![Vfn](https://github.com/carlomigs/btgym/blob/master/examples/img/v_fn_ep.png?raw=1)

##### - visualisations of hidden state of LSTM blocks:
- 256 cells along vertical axis, environment timesteps - along horisontal one; it can be clearly seen how rnn state gets reset when deal is closed.
![LSTM_state](https://github.com/carlomigs/btgym/blob/master/examples/img/lstm_h_ep.png?raw=1)

#### TODO:
- 1. Relax agent dependency on advised actions by annelaing `guided_lambda` in the course of training;
- 2. Implemnet potential-based approach to advising as in Wiewiora et al.

In [1]:
import warnings
warnings.filterwarnings("ignore") # suppress h5py deprecation warning

import os
import backtrader as bt
import numpy as np
from gym import spaces

from btgym import BTgymEnv, BTgymDataset, BTgymRandomDataDomain
from btgym.algorithms import Launcher

from btgym.research.gps.aac import GuidedAAC
from btgym.research.gps.policy import GuidedPolicy_0_0
from btgym.research.gps.strategy import GuidedStrategy_0_0, ExpertObserver


In [0]:
# Set backtesting engine and parameters:

engine = bt.Cerebro()

engine.addstrategy(
    GuidedStrategy_0_0,
    drawdown_call=10, # max % to loose, in percent of initial cash
    target_call=10,  # max % to win, same
    skip_frame=10,
    gamma=0.99,
    state_ext_scale=np.linspace(4e3, 1e3, num=6),
    reward_scale=7,
    expert_config=  # see btgym.research.gps.oracle.Oracle class for details
        {
            'time_threshold': 5,
            'pips_threshold': 10, 
            'pips_scale': 1e-4,
            'kernel_size': 10,
            'kernel_stddev': 1,
        },
)

# Expert actions observer:
engine.addobserver(ExpertObserver)

# Set leveraged account:
engine.broker.setcash(2000)
engine.broker.setcommission(commission=0.0001, leverage=10.0) # commisssion to imitate spread
engine.addsizer(bt.sizers.SizerFix, stake=5000)  

# Data: uncomment to get up to six month of 1 minute bars:
data_m1_6_month = [
    './data/DAT_ASCII_EURUSD_M1_201701.csv',
    './data/DAT_ASCII_EURUSD_M1_201702.csv',
    './data/DAT_ASCII_EURUSD_M1_201703.csv',
    #'./data/DAT_ASCII_EURUSD_M1_201704.csv',
    #'./data/DAT_ASCII_EURUSD_M1_201705.csv',
    #'./data/DAT_ASCII_EURUSD_M1_201706.csv',
]

# Uncomment single choice of source file:
dataset = BTgymRandomDataDomain(  
    #filename=data_m1_6_month,
    #filename='./data/DAT_ASCII_EURUSD_M1_2016.csv', # full year
    filename='./data/test_sine_1min_period256_delta0002.csv',  # simple sine 

    trial_params=dict(
        start_weekdays={0, 1, 2, 3, 4, 5, 6},
        sample_duration={'days': 3, 'hours': 0, 'minutes': 0},
        start_00=False,
        time_gap={'days': 1, 'hours': 10},
        test_period={'days': 0, 'hours': 0, 'minutes': 0},
    ),
    episode_params=dict(
        start_weekdays={0, 1, 2, 3, 4, 5, 6},
        sample_duration={'days': 1, 'hours': 23, 'minutes': 50},
        start_00=False,
        time_gap={'days': 1, 'hours': 0},
    ),
)

env_config = dict(
    class_ref=BTgymEnv, 
    kwargs=dict(
        dataset=dataset,
        engine=engine,
        render_modes=['episode', 'human', 'external', 'internal'],
        render_state_as_image=True,
        render_ylabel='OHL_diff. / Internals',
        render_size_episode=(12,8),
        render_size_human=(9, 4),
        render_size_state=(11, 3),
        render_dpi=75,
        port=5000,
        data_port=4999,
        connect_timeout=90,
        verbose=0,
    )
)

cluster_config = dict(
    host='127.0.0.1',
    port=12230,
    num_workers=4,  # Set according CPU's available or so
    num_ps=1,
    num_envs=1,
    log_dir=os.path.expanduser('~/tmp/gps'),
)

policy_config = dict(
    class_ref=GuidedPolicy_0_0,
    kwargs={
        'lstm_layers': (256, 256),
        'lstm_2_init_period': 50,
        'conv_2d_layer_config': (
             (32, (3, 1), (2, 1)),
             (32, (3, 1), (2, 1)),
             (64, (3, 1), (2, 1)),
             (64, (3, 1), (2, 1))
         ),
        'encode_internal_state': False,
    }
)

trainer_config = dict(
    class_ref=GuidedAAC,
    kwargs=dict(
        opt_learn_rate=1e-4, # scalar or random log-uniform 
        opt_end_learn_rate=1e-5,
        opt_decay_steps=20*10**6,
        model_gamma=0.99,
        model_gae_lambda=1.0,
        model_beta=0.01, # Entropy reg, scalar or random log-uniform
        aac_lambda=1.0, # main a3c loss weight
        guided_lambda=1.0,  # Imitation loss weight
        guided_decay_steps=10*10**6,  # annealing guided_lambda to zero in 10M steps
        rollout_length=20,
        time_flat=True,
        use_value_replay=False,
        episode_train_test_cycle=[1,0],
        model_summary_freq=100,
        episode_summary_freq=5,
        env_render_freq=5,
    )
)

In [3]:
launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    trainer_config=trainer_config,
    policy_config=policy_config,
    test_mode=False,
    max_env_steps=100*10**6,
    root_random_seed=0,
    purge_previous=1,  # ask to override previously saved model and logs
    verbose=0,
)

# Train it:
launcher.run()

[2019-09-09 23:06:58.134763] NOTICE: LauncherShell: </root/tmp/gps> created.






[2019-09-09 23:07:00.513264] ERROR: RndDataDomain_0: Data file <./data/test_sine_1min_period256_delta0002.csv> not specified / not found / parser error.


Process BTgymDataFeedServer-2:1:
Traceback (most recent call last):
  File "/content/btgym/btgym/dataserver.py", line 125, in run
    assert not self.dataset.data.empty
AttributeError: 'NoneType' object has no attribute 'empty'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/btgym/btgym/datafeed/base.py", line 438, in read_csv
    assert filename and os.path.isfile(filename)
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/content/btgym/btgym/dataserver.py", line 128, in run
    self.dataset.read_csv()
  File "/content/btgym/btgym/datafeed/base.py", line 462, in read_csv
    raise FileNotFoundError(msg)
FileNotFoundError: Data file <./data/test_sine_1min_period256_delta0002.csv> not specified / not found / parser error.












********************************************************************************************
**  Press `Ctrl-C` or jupyter:[Kernel]->[Interrupt] to stop training and close launcher.  **
********************************************************************************************

[2019-09-09 23:08:31.562676] ERROR: BTgymAPIshell_0: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
[2019-09-09 23:08:31.565404] ERROR: Worker_0: failed to make BTGym environment at port_5000.
Traceback (most recent call last):
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/content/btgym/btgym/envs/base.py", line 300, in __init__
    self._start_data_server()
  File "/content/btgym/btgym/envs/base.py", line 976, in _start_data_server
    raise ConnectionError(msg)
ConnectionError: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
[2019-09-09 23:08:31.571208] ERROR: Worker_0: Data_server unreachab

Process Worker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/content/btgym/btgym/algorithms/worker.py", line 403, in run
    raise e
  File "/content/btgym/btgym/algorithms/worker.py", line 283, in run
    raise e
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/content/btgym/btgym/envs/base.py", line 300, in __init__
    self._start_data_server()
  File "/content/btgym/btgym/envs/base.py", line 976, in _start_data_server
    raise ConnectionError(msg)
ConnectionError: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.


[2019-09-09 23:08:35.326890] ERROR: BTgymAPIshell_1: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
[2019-09-09 23:08:35.330757] ERROR: Worker_1: failed to make BTGym environment at port_5001.
Traceback (most recent call last):
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/content/btgym/btgym/envs/base.py", line 300, in __init__
    self._start_data_server()
  File "/content/btgym/btgym/envs/base.py", line 976, in _start_data_server
    raise ConnectionError(msg)
ConnectionError: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
[2019-09-09 23:08:35.334626] ERROR: BTgymAPIshell_2: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
[2019-09-09 23:08:35.336656] ERROR: Worker_2: failed to make BTGym environment at port_5002.
Traceback (most recent call last):
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/con

...

[2019-09-09 23:08:35.354119] ERROR: Worker_1: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
Traceback (most recent call last):
  File "/content/btgym/btgym/algorithms/worker.py", line 283, in run
    raise e
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/content/btgym/btgym/envs/base.py", line 300, in __init__
    self._start_data_server()
  File "/content/btgym/btgym/envs/base.py", line 976, in _start_data_server
    raise ConnectionError(msg)
ConnectionError: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.


Process Worker-3:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/content/btgym/btgym/algorithms/worker.py", line 403, in run
    raise e
  File "/content/btgym/btgym/algorithms/worker.py", line 283, in run
    raise e
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/content/btgym/btgym/envs/base.py", line 300, in __init__
    self._start_data_server()
  File "/content/btgym/btgym/envs/base.py", line 976, in _start_data_server
    raise ConnectionError(msg)
ConnectionError: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.


[2019-09-09 23:08:35.441349] ERROR: BTgymAPIshell_3: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
[2019-09-09 23:08:35.445089] ERROR: Worker_3: failed to make BTGym environment at port_5003.
Traceback (most recent call last):
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/content/btgym/btgym/envs/base.py", line 300, in __init__
    self._start_data_server()
  File "/content/btgym/btgym/envs/base.py", line 976, in _start_data_server
    raise ConnectionError(msg)
ConnectionError: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
[2019-09-09 23:08:35.449026] ERROR: Worker_3: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.
Traceback (most recent call last):
  File "/content/btgym/btgym/algorithms/worker.py", line 283, in run
    raise e
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/content/btgym/btgym/en

Process Worker-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/content/btgym/btgym/algorithms/worker.py", line 403, in run
    raise e
  File "/content/btgym/btgym/algorithms/worker.py", line 283, in run
    raise e
  File "/content/btgym/btgym/algorithms/worker.py", line 271, in run
    **env_kwargs
  File "/content/btgym/btgym/envs/base.py", line 300, in __init__
    self._start_data_server()
  File "/content/btgym/btgym/envs/base.py", line 976, in _start_data_server
    raise ConnectionError(msg)
ConnectionError: Data_server unreachable with status: <receive_failed_due_to_connect_timeout>.


[2019-09-09 23:42:04.337446] NOTICE: LauncherShell: worker_1 has joined.
[2019-09-09 23:42:04.344156] NOTICE: LauncherShell: worker_2 has joined.
[2019-09-09 23:42:04.346155] NOTICE: LauncherShell: worker_3 has joined.
[2019-09-09 23:42:04.351656] NOTICE: LauncherShell: chief_worker_0 has joined.
[2019-09-09 23:42:04.353815] NOTICE: LauncherShell: parameter_server_0 has joined.
[2019-09-09 23:42:04.354731] NOTICE: LauncherShell: Launcher closed.
